# Post-Classical Clean Up 4: Hyperion

In [ ]:
label = 'hyperion'

In [1]:
from datetime import datetime

import pandas
import sqlalchemy

SITE_URL = 'https://musicbrainz.org'

# import postgresql environ variables
# defined in postgres-dockerfile/postgres.env
PGHOST = '127.0.0.1'
PGDATABASE = 'musicbrainz'
PGUSER = 'musicbrainz'
PGPASSWORD = 'musicbrainz'
PGPORT = 5432

def sql(query, **kwargs):
    """helper function for SQL queries using the %(...) syntax
    Parameters for the query must be passed as keyword arguments
    e.g. sql('SELECT * FROM artist WHERE name=%(singer)s', singer='Bob Dylan')
    """
    engine = sqlalchemy.create_engine(
        'postgresql+psycopg2://'
        '{PGUSER}:{PGPASSWORD}@{PGHOST}:{PGPORT}/{PGDATABASE}'.format(**globals()),
        isolation_level='READ UNCOMMITTED')
    return pandas.read_sql(query, engine, params=kwargs)

# helper function to build canonical URLs
def _mb_link(entity_type, mbid):
    return '<a href="{url}/{entity_type}/{mbid}">{mbid}</a>'.format(
        url=SITE_URL, **locals())

mb_release_link = lambda mbid: _mb_link('release', mbid) # noqa

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [19]:
df = sql("""SELECT id, gid, name, comment FROM label WHERE name IN ('hyperion', 'helios');""")
label_mbid = df.gid.tolist()
df.head()

,id,gid,name,comment
0,388,08e6c3c8-81ab-405f-9cff-10f6b8db064c,hyperion,UK classical
1,22708,0a94e96a-9219-4dd7-a529-18d34e77f50f,helios,UK classical


In [42]:
label_mbid

[UUID('08e6c3c8-81ab-405f-9cff-10f6b8db064c'),
 UUID('0a94e96a-9219-4dd7-a529-18d34e77f50f')]

## New releases added after Feb. 1st

Actually I'm checking when "release label" links were added to release, which is most of the time done together.

In [43]:
df = sql("""
SELECT editor.name AS editor,
       COUNT(*) AS edit_count
  FROM label
  JOIN edit_label  ON label.id = edit_label.label
  JOIN edit        ON edit.id = edit_label.edit
  JOIN editor      ON editor.id = edit.editor
 WHERE label.gid IN ('08e6c3c8-81ab-405f-9cff-10f6b8db064c', '0a94e96a-9219-4dd7-a529-18d34e77f50f')
   AND edit.open_time >= '2018-02-01'
GROUP BY editor.name
ORDER BY edit_count DESC
   ;""")
df

,editor,edit_count
0,loujin,74
1,ListMyCDs.com,38
2,reosarevok,35
3,mmirG,19
4,MetaTunes,14
5,steinbdj,9
6,CatCat,6
7,Griomo,5
8,stupidname,3
9,mhendu,3


In [44]:
df.edit_count.sum()

213

## Recording edits

Find all edits between Feb. 1st and March 14th regarding recordings present on Hyperion/Helios releases

In [45]:
df = sql("""
SELECT editor.name AS editor,
       COUNT(*) AS edit_count
  FROM recording     AS rec
  JOIN edit_recording      ON rec.id = edit_recording.recording
  JOIN edit                ON edit.id = edit_recording.edit
  JOIN editor              ON editor.id = edit.editor
  JOIN track               on track.recording = rec.id
  JOIN medium        AS m  ON track.medium = m.id
  JOIN release       AS r  ON m.release = r.id
  JOIN release_label AS rl ON rl.release = r.id
  JOIN label               ON rl.label = label.id
 WHERE label.gid IN ('08e6c3c8-81ab-405f-9cff-10f6b8db064c', '0a94e96a-9219-4dd7-a529-18d34e77f50f')
   AND edit.open_time >= '2018-02-01'
GROUP BY editor.name
ORDER BY edit_count DESC
   ;""")
df

,editor,edit_count
0,loujin,20189
1,ListMyCDs.com,11245
2,reosarevok,10609
3,stupidname,6652
4,CatCat,4097
5,dosoe,3513
6,obtext,2943
7,steinbdj,1330
8,MetaTunes,1024
9,Griomo,807


## Release covers added

In [41]:
df = sql("""
SELECT COUNT(*) AS edit_count
  FROM release        AS r
  JOIN edit_release        ON r.id = edit_release.release
  JOIN edit                ON edit.id = edit_release.edit
  JOIN release_label AS rl ON rl.release = r.id
  JOIN label               ON rl.label = label.id
 WHERE label.gid IN ('08e6c3c8-81ab-405f-9cff-10f6b8db064c', '0a94e96a-9219-4dd7-a529-18d34e77f50f')
   AND edit.open_time >= '2018-02-01'
   AND edit.type = 314
   ;""")

df

,edit_count
0,377


## phonographic relationships added on recordings

In [51]:
sql("""
SELECT COUNT(*) AS edit_count
  FROM recording     AS r
  JOIN edit_recording      ON r.id = edit_recording.recording
  JOIN edit                ON edit.id = edit_recording.edit
  JOIN l_label_recording AS lar ON r.id = lar.entity1
  JOIN label                    ON label.id = lar.entity0
  JOIN link                     ON link.id = lar.link
 WHERE label.gid IN ('08e6c3c8-81ab-405f-9cff-10f6b8db064c', '0a94e96a-9219-4dd7-a529-18d34e77f50f')
   AND edit.open_time >= '2018-02-01'
   AND link.link_type = 867
;""")

,edit_count
0,315
